## Section 1: Similarity Computations

### 1.a Jaccard Similarity (Users)

### 1.b Jaccard Similarity (Movies)

### 1.c Pearson Correlation (Users)

## Section 2: Collaborative Filtering

### 2.d Application to Real Data

## Section 3: Latent Factor Recommendation

### 3.b Implementation and Evaluation